In [108]:
#원본 데이터 txt 파일 불러오기
import pandas as pd
import numpy as np
df_ = pd.read_csv("2018_서울시_국민기초생활보장_수급자.txt", sep = "\t", header = 1)

In [109]:
df_.columns

Index(['기간', '자치구', '동', '가구', '시설', '인원', '가구.1', '인원.1', '가구.2', '인원.2',
       '가구.3', '인원.3', '시설.1', '인원.4'],
      dtype='object')

In [110]:
df_

,기간,자치구,동,가구,시설,인원,가구.1,인원.1,가구.2,인원.2,가구.3,인원.3,시설.1,인원.4
0,2018,합계,합계,"197,278",604,"290,222","169,271","227,396","23,360","44,271","4,647","6,324",604,"12,231"
1,2018,본청,본청,-,8,880,-,-,-,-,-,-,8,880
2,2018,종로구,소계,"3,109",14,"4,450","2,821","3,403",216,318,72,80,14,649
3,2018,종로구,사직동,99,-,121,96,116,3,5,-,-,-,-
4,2018,종로구,삼청동,34,-,48,25,31,7,14,2,3,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,2018,강동구,둔촌2동,281,-,407,240,320,38,84,3,3,-,-
471,2018,강동구,암사1동,931,-,"1,414",791,"1,123",116,249,24,42,-,-
472,2018,강동구,천호2동,"1,009",-,"1,376",871,"1,131",117,218,21,27,-,-
473,2018,강동구,길동,707,-,"1,036",593,804,98,213,16,19,-,-


In [111]:
#불필요한 column drop
df = df_.drop(['기간', '가구', '시설', '가구.1', '인원.1', '가구.2', '인원.2',
               '가구.3', '인원.3', '시설.1', '인원.4'], axis = 1)

In [112]:
df

,자치구,동,인원
0,합계,합계,"290,222"
1,본청,본청,880
2,종로구,소계,"4,450"
3,종로구,사직동,121
4,종로구,삼청동,48
...,...,...,...
470,강동구,둔촌2동,407
471,강동구,암사1동,"1,414"
472,강동구,천호2동,"1,376"
473,강동구,길동,"1,036"


In [113]:
#불필요한 row 제거
df = df[df['동'] != '소계']
df = df[df['동'] != '합계']
df = df[df['동'] != '기타']
df = df[df['동'] != '본청']

In [114]:
#결측치 -에서 0으로 바꾸기(문자면 계산 시 오류)
df = df.replace(to_replace ='-', value = '0', regex = True)

In [115]:
#geo-json에 맞춰서 동 이름 변환
df = df.replace(to_replace ='종로5·6가동', value = '종로5.6가동', regex = True)

In [116]:
#결측된 둔촌 1동을 둔촌2동 데이터로 보정
df = df.append({'자치구': '강동구', '동':'둔촌1동', '인원':407}, ignore_index=True)

In [117]:
df

,자치구,동,인원
0,종로구,사직동,121
1,종로구,삼청동,48
2,종로구,부암동,141
3,종로구,평창동,124
4,종로구,무악동,137
...,...,...,...
419,강동구,둔촌2동,407
420,강동구,암사1동,"1,414"
421,강동구,천호2동,"1,376"
422,강동구,길동,"1,036"


In [118]:
#규칙에 맞게 column 이름 재지정
df.columns = ['구', '동', '행정동별기초생활보장수급자합계']

In [119]:
#행정동코드 merge를 위한 xlsx 파일 로드
data = pd.read_excel("geojson_행정동코드.xlsx")

In [120]:
#행정동코드 merge
df= pd.merge(df, data, on=["구", "동"] ,  how='right')

In [121]:
#column 순서 재배치
df = df[['행정동코드', '구', '동', '행정동별기초생활보장수급자합계']]

In [122]:
#전처리한 데이터를 xlsx 로 저장
df.to_excel('기초생활보장수급자.xlsx', encoding = 'utf-8')

In [123]:
df

,행정동코드,구,동,행정동별기초생활보장수급자합계
0,11110530,종로구,사직동,121
1,11110540,종로구,삼청동,48
2,11110550,종로구,부암동,141
3,11110560,종로구,평창동,124
4,11110570,종로구,무악동,137
...,...,...,...,...
419,11740700,강동구,둔촌2동,407
420,11740570,강동구,암사1동,"1,414"
421,11740610,강동구,천호2동,"1,376"
422,11740685,강동구,길동,"1,036"
